In [16]:
import pandas as pd
from itertools import combinations, product

# Load data
dcmp_df = pd.read_csv("2025 DCMP - Match Data.csv")
csv7034_df = pd.read_csv("7034.csv")

# Normalize
dcmp_df['team_number'] = dcmp_df['team_key'].str.extract(r'(\\d+)$').astype(float)
dcmp_df['match_key'] = dcmp_df['match_number'].astype(str) + '_' + dcmp_df['comp_level']
csv7034_df = csv7034_df.dropna(subset=['Match #'])
csv7034_df['match_number'] = csv7034_df['Match #'].astype(int)
csv7034_df['team_number'] = csv7034_df['Team #']
csv7034_df['match_key'] = csv7034_df['match_number'].astype(str) + '_qm'
csv7034_df['alliance'] = csv7034_df['Robot'].str[0].map({'R': 'red', 'B': 'blue'})

# Step 1: Extract alliance info from DCMP
def extract_alliance_teams(df):
    alliances = {}
    for _, row in df.iterrows():
        key = f"{row['match_number']}_{row['comp_level']}"
        if key not in alliances:
            alliances[key] = {
                'red': [row['r1'], row['r2'], row['r3']],
                'blue': [row['b1'], row['b2'], row['b3']],
            }
    return alliances

match_alliances = extract_alliance_teams(dcmp_df)

def assign_alliance(row):
    key = row['match_key']
    team_key = row['team_key']
    alliance = match_alliances.get(key, {})
    if 'red' in alliance and team_key in alliance['red']:
        return 'red'
    elif 'blue' in alliance and team_key in alliance['blue']:
        return 'blue'
    return None

# Now assign it
dcmp_df['alliance'] = dcmp_df.apply(assign_alliance, axis=1)

def total_mixed_coral(team_rows, prefix):
    return sum(
        row.get(f'{prefix}_L1', 0) +
        row.get(f'{prefix}_L2', 0) +
        row.get(f'{prefix}_L3', 0) +
        row.get(f'{prefix}_L4', 0)
        for _, row in team_rows
    )

def find_best_cell_mixed_combo(dcmp_rows, csv_rows, target, prefix):
    team_data = {}
    for row in dcmp_rows + csv_rows:
        team_number = row['team_number']
        if team_number not in team_data:
            team_data[team_number] = {}
        source = 'dcmp' if f'{prefix}_L1' in row else '7034'
        team_data[team_number][source] = row

    best_diff = float('inf')
    best_combo = None

    team_numbers = list(team_data.keys())
    for team_set in combinations(team_numbers, 3):
        cell_mix_combos = list(product(['dcmp', '7034'], repeat=4))
        all_robot_mixes = list(product(cell_mix_combos, repeat=3))  # 4096

        for robot_mix in all_robot_mixes:
            valid = True
            combo = []
            total = 0

            for team, mix in zip(team_set, robot_mix):
                sources = team_data[team]
                mixed_row = {'team_number': team}
                row_sum = 0
                for i, lvl in enumerate(['L1', 'L2', 'L3', 'L4']):
                    dcmp_col = f'{prefix}_L{lvl[-1]}'
                    if mix[i] == 'dcmp' and 'dcmp' in sources:
                        val = sources['dcmp'].get(dcmp_col, 0)
                    elif mix[i] == '7034' and '7034' in sources:
                        val = sources['7034'].get(f'{prefix}L{lvl[-1]}', 0)
                    else:
                        valid = False
                        break
                    mixed_row[dcmp_col] = val
                    row_sum += val
                if not valid:
                    break
                total += row_sum
                combo.append((team, mixed_row))

            if valid and abs(total - target) < best_diff:
                best_diff = abs(total - target)
                best_combo = combo

    return best_combo if best_combo else []

# Process all matches
all_results = []

for match_key in dcmp_df['match_key'].unique():
    for alliance in ['red', 'blue']:
        dcmp_rows = list(dcmp_df[(dcmp_df['match_key'] == match_key) & (dcmp_df['alliance'] == alliance)].to_dict('records'))
        csv_rows = list(csv7034_df[(csv7034_df['match_key'] == match_key) & (csv7034_df['alliance'] == alliance)].to_dict('records'))
        if not dcmp_rows and not csv_rows:
            continue

        auto_target = next((r['ba_autoCoralCount'] for r in dcmp_rows if pd.notna(r.get('ba_autoCoralCount'))), 0)
        tele_target = next((r['ba_teleopCoralCount'] for r in dcmp_rows if pd.notna(r.get('ba_teleopCoralCount'))), 0)

        best_auto = find_best_cell_mixed_combo(dcmp_rows, csv_rows, auto_target, 'a')
        best_tele = find_best_cell_mixed_combo(dcmp_rows, csv_rows, tele_target, 't')

        team_info = {r['team_number']: r for r in dcmp_rows}

        final_teams = set()
        for team, r in best_auto + best_tele:
            if team not in final_teams:
                full = team_info.get(team, {}).copy()
                full.update(r)
                full['match_key'] = match_key
                full['alliance'] = alliance
                full['selected_for_auto'] = 'auto' if any(x for x in best_auto if x[0] == team) else None
                full['selected_for_teleop'] = 'teleop' if any(x for x in best_tele if x[0] == team) else None
                all_results.append(full)
                final_teams.add(team)

# Save to CSV
df_final = pd.DataFrame(all_results)
df_final = df_final.sort_values(by=['match_key', 'alliance', 'team_number'])
df_final.to_csv("smart_cell_mixed_output.csv", index=False)
print("Done! Saved to smart_cell_mixed_output.csv")


Done! Saved to smart_cell_mixed_output.csv


In [1]:
import pandas as pd
from itertools import combinations, product

# --- Load and normalize data ---
dcmp_df = pd.read_csv("2025 DCMP - Match Data.csv")
csv7034_df = pd.read_csv("7034.csv")

# Extract team numbers and match keys
dcmp_df['team_number'] = dcmp_df['team_key'].str.extract(r'(\d+)$').astype(float)
dcmp_df['match_key'] = dcmp_df['match_number'].astype(str) + '_' + dcmp_df['comp_level']

csv7034_df = csv7034_df.dropna(subset=['Match #'])
csv7034_df['match_number'] = csv7034_df['Match #'].astype(int)
csv7034_df['team_number'] = csv7034_df['Team #']
csv7034_df['match_key'] = csv7034_df['match_number'].astype(str) + '_qm'
csv7034_df['alliance'] = csv7034_df['Robot'].str[0].map({'R': 'red', 'B': 'blue'})

# --- Assign alliances to DCMP data ---
def extract_alliance_teams(df):
    alliances = {}
    for _, row in df.iterrows():
        key = f"{row['match_number']}_{row['comp_level']}"
        if key not in alliances:
            alliances[key] = {
                'red': [row['r1'], row['r2'], row['r3']],
                'blue': [row['b1'], row['b2'], row['b3']],
            }
    return alliances

match_alliances = extract_alliance_teams(dcmp_df)

def assign_alliance(row):
    key = row['match_key']
    team_key = row['team_key']
    alliance = match_alliances.get(key, {})
    if 'red' in alliance and team_key in alliance['red']:
        return 'red'
    elif 'blue' in alliance and team_key in alliance['blue']:
        return 'blue'
    return None

dcmp_df['alliance'] = dcmp_df.apply(assign_alliance, axis=1)

# --- Mixing logic ---
def find_best_cell_mixed_combo(dcmp_rows, csv_rows, target, prefix):
    team_data = {}
    for row in dcmp_rows + csv_rows:
        team_number = row['team_number']
        if team_number not in team_data:
            team_data[team_number] = {}
        source = 'dcmp' if f'{prefix}_L1' in row else '7034'
        team_data[team_number][source] = row

    best_diff = float('inf')
    best_combo = None
    team_numbers = list(team_data.keys())

    for team_set in combinations(team_numbers, 3):
        cell_mix_combos = list(product(['dcmp', '7034'], repeat=4))
        all_robot_mixes = product(cell_mix_combos, repeat=3)

        for robot_mix in all_robot_mixes:
            valid = True
            combo = []
            total = 0

            for team, mix in zip(team_set, robot_mix):
                sources = team_data[team]
                mixed_row = {'team_number': team}
                row_sum = 0
                for i, lvl in enumerate(['L1', 'L2', 'L3', 'L4']):
                    dcmp_col = f'{prefix}_L{lvl[-1]}'
                    if mix[i] == 'dcmp' and 'dcmp' in sources:
                        val = sources['dcmp'].get(dcmp_col, 0)
                    elif mix[i] == '7034' and '7034' in sources:
                        val = sources['7034'].get(f'{prefix}L{lvl[-1]}', 0)
                    else:
                        valid = False
                        break
                    mixed_row[dcmp_col] = val
                    row_sum += val
                if not valid:
                    break
                total += row_sum
                combo.append((team, mixed_row))

            if valid and abs(total - target) < best_diff:
                best_diff = abs(total - target)
                best_combo = combo

    return best_combo if best_combo else []

# --- Merge 7034 coral into DCMP base row ---
def apply_7034_coral_to_dcmp(dcmp_row, row_7034, prefix):
    updated = dcmp_row.copy()
    for lvl in ['L1', 'L2', 'L3', 'L4']:
        dcmp_col = f'{prefix}_L{lvl[-1]}'
        col_7034 = f'{prefix}L{lvl[-1]}'
        if col_7034 in row_7034:
            updated[dcmp_col] = row_7034[col_7034]
    return updated

# --- Main smart merge loop ---
results = []
for match_key in dcmp_df['match_key'].unique():
    for alliance in ['red', 'blue']:
        dcmp_rows = list(dcmp_df[(dcmp_df['match_key'] == match_key) & (dcmp_df['alliance'] == alliance)].to_dict('records'))
        csv_rows = list(csv7034_df[(csv7034_df['match_key'] == match_key) & (csv7034_df['alliance'] == alliance)].to_dict('records'))

        if not dcmp_rows:
            continue

        team_info = {row['team_number']: row for row in dcmp_rows}
        auto_target = next((r['ba_autoCoralCount'] for r in dcmp_rows if pd.notna(r.get('ba_autoCoralCount'))), 0)
        tele_target = next((r['ba_teleopCoralCount'] for r in dcmp_rows if pd.notna(r.get('ba_teleopCoralCount'))), 0)

        best_auto = find_best_cell_mixed_combo(dcmp_rows, csv_rows, auto_target, 'a')
        best_tele = find_best_cell_mixed_combo(dcmp_rows, csv_rows, tele_target, 't')

        seen = set()
        # Group selected rows by team number for clarity
        selected_rows = {}
        for team, row in best_auto:
            if team not in selected_rows:
                selected_rows[team] = {'a': row}
            else:
                selected_rows[team]['a'] = row
        for team, row in best_tele:
            if team not in selected_rows:
                selected_rows[team] = {'t': row}
            else:
                selected_rows[team]['t'] = row

        # Merge 7034 coral into DCMP base row
        for team, parts in selected_rows.items():
            base = team_info.get(team, {}).copy()

            if 'a' in parts:
                base = apply_7034_coral_to_dcmp(base, parts['a'], 'a')
                base['selected_for_auto'] = 'auto'
            if 't' in parts:
                base = apply_7034_coral_to_dcmp(base, parts['t'], 't')
                base['selected_for_teleop'] = 'teleop'

            base['match_key'] = match_key
            base['alliance'] = alliance
            results.append(base)


# --- Sort results by match number with red alliance first, then blue ---
smart_df = pd.DataFrame(results)

# Extract the numeric match number from match_key for proper sorting
smart_df['match_number_sort'] = smart_df['match_key'].str.extract(r'(\d+)').astype(int)

# Create alliance sort order (red=0, blue=1 for sorting)
smart_df['alliance_sort'] = smart_df['alliance'].map({'red': 0, 'blue': 1})

# Sort by match number (numeric), then alliance (red first), then team number
smart_df = smart_df.sort_values(by=['match_number_sort', 'alliance_sort', 'team_number'])

# Remove helper columns before saving
smart_df = smart_df.drop(columns=['match_number_sort', 'alliance_sort'])

#print(smart_df)
smart_df.to_csv("smart_cell_mixed_output2.csv", index=False)

print("Saved to smart_cell_mixed_output2.csv")


Saved to smart_cell_mixed_output2.csv
